# LAS to COPC (Cloud-Optimized Point Cloud) Conversion

This notebook demonstrates converting LAS files to COPC format using PDAL.

In [1]:
# Import required libraries
from pathlib import Path
import subprocess
from calipso_tool.las_to_copc import las_to_copc, las_to_copc_pipeline, batch_las_to_copc

## Single File Conversion

In [2]:
# Find LAS files in current directory
las_files = list(Path(".").glob("*.las"))

if las_files:
    print(f"Found {len(las_files)} LAS files:")
    for f in las_files:
        print(f"  - {f.name} ({f.stat().st_size / (1024*1024):.2f} MB)")
else:
    print("No LAS files found in current directory")
    print("Please run the HDF to LAS conversion first.")

Found 11 LAS files:
  - points.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2018-12N.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-04N.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-03N.las (48.56 MB)
  - custom_scaled.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-01D.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-02N.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-01N.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-03D.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-02D.las (48.56 MB)
  - CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2018-12D.las (48.56 MB)


In [3]:
# Convert first LAS file to COPC
if las_files:
    las_file = las_files[0]
    
    try:
        print(f"Converting {las_file.name} to COPC format...")
        copc_file = las_to_copc_pipeline(las_file)
        
        # Compare file sizes
        las_size = las_file.stat().st_size / (1024*1024)
        copc_size = copc_file.stat().st_size / (1024*1024)
        compression = (1 - copc_size/las_size) * 100
        
        print(f"\nConversion complete!")
        print(f"  Original LAS: {las_size:.2f} MB")
        print(f"  COPC output: {copc_size:.2f} MB")
        print(f"  Compression: {compression:.1f}%")
        
    except Exception as e:
        print(f"Conversion failed: {e}")

Converting points.las to COPC format...
Converting points.las to COPC format...
✓ Created: points.copc.laz

File sizes:
  Input LAS: 48.56 MB
  Output COPC: 3.94 MB
  Compression: 91.9%

Conversion complete!
  Original LAS: 48.56 MB
  COPC output: 3.94 MB
  Compression: 91.9%


## Batch Conversion

In [4]:
# Convert all LAS files in directory
if len(las_files) > 1:
    print(f"Converting {len(las_files)} LAS files to COPC format...\n")
    
    successful, failed = batch_las_to_copc(
        directory=".",
        pattern="*.las",
        skip_existing=True
    )
    
    if successful:
        print("\nSuccessfully converted:")
        for copc in successful:
            print(f"  ✓ {copc.name}")
    
    if failed:
        print("\nFailed conversions:")
        for las, error in failed:
            print(f"  ✗ {las.name}: {error}")

Converting 11 LAS files to COPC format...

Found 11 LAS files to convert
⏭️  Skipping points.las (COPC already exists)
Converting CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2018-12N.las to COPC format...
✓ Created: CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2018-12N.copc.laz

File sizes:
  Input LAS: 48.56 MB
  Output COPC: 5.56 MB
  Compression: 88.6%
Converting CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-04N.las to COPC format...
✓ Created: CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-04N.copc.laz

File sizes:
  Input LAS: 48.56 MB
  Output COPC: 5.56 MB
  Compression: 88.6%
Converting CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-03N.las to COPC format...
✓ Created: CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-03N.copc.laz

File sizes:
  Input LAS: 48.56 MB
  Output COPC: 5.56 MB
  Compression: 88.6%
Converting custom_scaled.las to COPC format...
✓ Created: custom_scaled.copc.laz

File sizes:
  Input LAS: 48.56 MB
  Output 

## Inspect COPC Files

In [5]:
# List all COPC files and their properties
copc_files = list(Path(".").glob("*.copc.laz"))

if copc_files:
    print(f"COPC files in directory:\n")
    
    total_las_size = 0
    total_copc_size = 0
    
    for copc in copc_files:
        # Try to find corresponding LAS file
        las_name = copc.name.replace('.copc.laz', '.las')
        las_file = copc.parent / las_name
        
        copc_size = copc.stat().st_size / (1024*1024)
        total_copc_size += copc_size
        
        print(f"{copc.name}:")
        print(f"  Size: {copc_size:.2f} MB")
        
        if las_file.exists():
            las_size = las_file.stat().st_size / (1024*1024)
            total_las_size += las_size
            compression = (1 - copc_size/las_size) * 100
            print(f"  Compression: {compression:.1f}% (from {las_size:.2f} MB)")
        
        # Get point count using PDAL
        result = subprocess.run(
            ["pdal", "info", str(copc), "--summary"],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0 and "count" in result.stdout:
            # Extract point count from output
            import json
            try:
                info = json.loads(result.stdout)
                if "summary" in info and "points" in info["summary"]:
                    points = info["summary"]["points"]
                    print(f"  Points: {points:,}")
            except:
                pass
        
        print()
    
    if total_las_size > 0:
        total_compression = (1 - total_copc_size/total_las_size) * 100
        print(f"\nTotal compression: {total_compression:.1f}%")
        print(f"  Original: {total_las_size:.2f} MB")
        print(f"  Compressed: {total_copc_size:.2f} MB")
else:
    print("No COPC files found in current directory")

COPC files in directory:

CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-01D.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-01N.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2018-12D.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2018-12N.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

custom_scaled.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-03D.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-03N.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2019-04N.copc.laz:
  Size: 5.56 MB
  Compression: 88.6% (from 48.56 MB)

points.copc.laz:
  Size: 3.9

## Advanced: Custom COPC Creation

Create COPC with custom pipeline parameters.

In [ ]:
# Example: Create custom PDAL pipeline for COPC
import json

if las_files:
    las_file = las_files[0]
    
    # Create custom pipeline with filters
    custom_pipeline = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": str(las_file)
            },
            {
                "type": "filters.outlier",
                "method": "statistical",
                "mean_k": 12,
                "multiplier": 2.0
            },
            {
                "type": "writers.copc",
                "filename": str(las_file.parent / f"{las_file.stem}_filtered.copc.laz")
            }
        ]
    }
    
    # Save and run custom pipeline
    pipeline_path = Path("custom_copc_pipeline.json")
    with open(pipeline_path, 'w') as f:
        json.dump(custom_pipeline, f, indent=2)
    
    print(f"Created custom pipeline: {pipeline_path}")
    print("\nThis pipeline includes outlier filtering before COPC creation.")
    print("\nTo run it:")
    print(f"pdal pipeline {pipeline_path}")

## Command Line Usage

You can also use the LAS to COPC converter from the command line:

```bash
# Single file conversion
python -m calipso_tool.las_to_copc input.las

# With custom output name
python -m calipso_tool.las_to_copc input.las -o output.copc.laz

# Using custom pipeline
python -m calipso_tool.las_to_copc input.las -p custom_pipeline.json

# Batch conversion
python -m calipso_tool.las_to_copc . --batch --pattern "*.las"
```

## COPC Benefits

Cloud-Optimized Point Clouds offer several advantages:

1. **Efficient Storage**: LAZ compression reduces file size by 50-90%
2. **Streaming**: Hierarchical structure allows efficient data access
3. **Cloud-Ready**: Optimized for HTTP range requests
4. **Wide Support**: Compatible with modern point cloud software
5. **Metadata**: Embedded spatial indexing and statistics

COPC files can be:
- Hosted on S3, Azure, or GCS for direct streaming
- Visualized in web browsers with Potree or Plasio
- Processed efficiently with PDAL or QGIS
- Integrated into 3D GIS workflows